In [1]:
"""
This is a CNN which is used in sentiment classification, idea of Kim's CNN.
Teng Li
21.09.2021
"""

"\nThis is a CNN which is used in sentiment classification, idea of Kim's CNN.\nTeng Li\n21.09.2021\n"

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset
import gensim
# then we can load word2vec
model = gensim.models.KeyedVectors.load_word2vec_format('/home/teng/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz',
                                                        binary=True,limit=10000)
#first add 'unk' to our model
model['unk'] = np.zeros(300,dtype=np.float32)
# the weights of Embedding layer are
W2V_weights = torch.FloatTensor(model.vectors)
# the vocab of this Embedding is
Vocab = model.wv.vocab

BATCH_SIZE = 2
EPOCHS = 5
WORD_VECTOR_SIZE = 300 # The size of word vector
FILTER_SIZE = [3,4,5] # 3 different size of filter
FILTER_NUM = [100,100,100] # The number of each size filter
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use GPU if available


class ConvNet(nn.Module):# inherit from nn.Module
    
    def __init__(self):#init the module
        super().__init__()
        #Embedding layer
        self.embed = nn.Embedding.from_pretrained(W2V_weights)
        #Conv layers
        self.conv1 = nn.Conv2d(1,FILTER_NUM[0],(FILTER_SIZE[0],WORD_VECTOR_SIZE))
        self.conv2 = nn.Conv2d(1,FILTER_NUM[1],(FILTER_SIZE[1],WORD_VECTOR_SIZE))
        self.conv3 = nn.Conv2d(1,FILTER_NUM[2],(FILTER_SIZE[2],WORD_VECTOR_SIZE))
        #Fc layer
        self.fc1 = nn.Linear(3*100, 80) #input 20*10*10, out 500
        self.fc2 = nn.Linear(80, 20)
        self.fc3 = nn.Linear(20, 2)
        
        
    def forward(self,x):#forward propagation
        #Embedding layer
        x = self.embed(x)   # batch*1*N(word_num in doc) -> batch*1*N*WORD_VECTOR_SIZE 
        print(x.shape)
        #Conv layers
        out1 = self.conv1(x) # batch*1*N*WORD_VECTOR_SIZE -> batch*100*N*WORD_VECTOR_SIZE
        out2 = self.conv2(x)
        out3 = self.conv3(x)
        out1 = torch.tanh(out1)
        out2 = torch.tanh(out2)
        out3 = torch.tanh(out3)
        
        #Max over time pooling layer
        max1,_ = torch.max(out1,2)
        max2,_ = torch.max(out2,2)
        max3,_ = torch.max(out3,2)
        #print(max1.size())
        
        #Concatenates the features and reshape for FC layer
        Max = torch.cat((max1,max2,max3),1)
        #print(Max.size())
        Max = Max.view(BATCH_SIZE,-1)
        #print(Max.size())
        
        #Fc layer
        Out = self.fc1(Max) # batch*300 -> batch*80
        Out = torch.tanh(Out)
        Out = self.fc2(Out) # batch*80 -> batch*20
        Out = torch.tanh(Out)
        Out = self.fc3(Out) # batch*20 -> batch*2
        Out = f.log_softmax(Out,dim=1) #"dim=1" means logsoftmax along 2 sentiment(pos or neg)

        return Out

    def predict(self,x):
        with torch.no_grad():
            out = self.forward(x)
            _,y_hat = torch.max()

SyntaxError: invalid syntax (<ipython-input-1-9c2bf544aaa9>, line 78)

In [63]:
# N is the number of word in doc
N = 100
data = torch.ByteTensor(BATCH_SIZE,1,N)
data = data.long()
print(data)

tensor([[[ 16,  45,  32, 180, 186, 127,   0,   0,  16,  45,  32, 180, 186, 127,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,  32,  33, 223, 201,  61,  86,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,  24, 211, 223, 201,  61,  86,
            0,   0,  32, 211, 223, 201,  61,  86,   0,   0,  64, 211, 223, 201,
           61,  86,   0,   0, 100,   0,   0,   0,   0,   0,   0,   0,  44,   1,
            0,   0]],

        [[  0,   0,   0,   0,  44,   1,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           88, 211, 223, 201,  61,  86,   0,   0,  96, 211, 223, 201,  61,  86,
            0,   0, 128, 211, 223, 201,  61,  86,   0,   0,   1,   0,   0,   0,
            0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   1,   0,
            0,   

In [57]:
data = torch.LongTensor([[[1,2,3,6,88,32]],
                     [[4,6,3,6,8,21]]])
print(data.shape)

torch.Size([2, 1, 6])


In [64]:
Text_CNN = ConvNet().to(DEVICE)
Out = Text_CNN(data)
print(Out)

torch.Size([2, 1, 100, 300])
tensor([[-0.7831, -0.6106],
        [-0.7824, -0.6112]], grad_fn=<LogSoftmaxBackward>)


(300,)